# Chapter 7: Data Generation, Conversion, and Manipulation

SQL has the following string features
    * Generation with `CHAR(UINT)`,
    * Manipulation with `POSITION(varchar IN varchar)`,
    * Location with `LOCATE(search, reference, location)`,
    * Comparison with `STRCMP(left, right)`,
    * Concatenation with `CONCAT(string1, string2, ..., stringN)`

There are numeric functions like `POW(X, Y)`, ceil and floor, truncation using `TRUNCATE(value, precision)`

There are datetime functions like `CAST(value AS TYPE)` and date difference using `DATEDIFF(date1, date2)`

In [1]:
import os

from dotenv import load_dotenv
from sqlalchemy import create_engine, URL, select, func
from sqlalchemy.orm import Session
import pandas as pd

from utils import print_sql_statement


load_dotenv()

url_object = URL.create(
    os.environ["DB_ENGINE"],
    username=os.environ["DB_USER"],
    password=os.environ["DB_PASSWD"],
    host=os.environ["DB_HOST"],
    database=os.environ["DB_NAME"],
)

engine = create_engine(url_object)

# Exercise

## 7-1
Write a query that returns the 17th through 25th characters of the sting "Please find the substring in this string"

## 7-2
Write a query that returns the absolute value and sign of the number -25.76823. Also return the number rounded to the nearest hundredth

## 7-3
Write a query to return just the month portion of the current date.

In [8]:
# 7-1
from sqlalchemy import func, text


with Session(engine) as session:
    df = pd.read_sql_query(
        """
            SELECT
                SUBSTRING('Please find the substring in this string', 17, 25) chars
            ;
        """,
        con=session.connection()
    )

    statement = (
        select(
            func.substring(text("'Please find the substring in this string'"), 17, 25)
        )
    )
    print_sql_statement(statement)
    results = session.execute(statement).all()

print(df)
print(results)

"""SELECT substring('Please find the substring in this string', :substring_2, :substring_3) AS substring_1"""
                      chars
0  substring in this string
[('substring in this string',)]


In [14]:
# 7-2

with Session(engine) as session:
    df = pd.read_sql_query(
        """
            SELECT
                ABS(-25.76823) absv
                , SIGN(-25.76823) sgn
                , TRUNCATE(-25.76823, 2) trun
                , ROUND(-25.76823, 2) rnd
            ;
        """,
        con=session.connection()
    )

    statement = (
        select(
            func.abs(-25.76823),
            func.sign(-25.76823),
            func.truncate(-25.76823, 2),
            func.round(-25.76823, 2)
        )
    )
    results = session.execute(statement).all()

print(df)
print(results)

       absv  sgn   trun    rnd
0  25.76823   -1 -25.76 -25.77
[(Decimal('25.76823'), -1, Decimal('-25.76'), Decimal('-25.77'))]


In [28]:
# 7-3
# Let's change the prompt to the month of the employee's start date
from model import Employee


with Session(engine) as session:
    df = pd.read_sql_query(
        """
            SELECT
                fname
                , lname
                , EXTRACT(MONTH FROM start_date) start_month
            FROM
                employee
            ;
        """,
        con=session.connection()
    )

    statement = (
        select(
            Employee.fname,
            Employee.lname,
            func.extract("MONTH", Employee.start_date)
        )
    )
    results = session.execute(statement).all()
    new_results = [
        (emp.fname, emp.lname, emp.start_date.month)
        for emp in session.query(Employee)
    ]

print(df)
print(results)
assert set(results) == set(new_results)

       fname      lname  start_month
0    Michael      Smith            6
1      Susan     Barker            9
2     Robert      Tyler            2
3      Susan  Hawthorne            4
4       John    Gooding           11
5      Helen    Fleming            3
6      Chris     Tucker            9
7      Sarah     Parker           12
8       Jane   Grossman            5
9      Paula    Roberts            7
10    Thomas    Ziegler           10
11  Samantha    Jameson            1
12      John      Blake            5
13     Cindy      Mason            8
14     Frank    Portman            4
15   Theresa    Markham            3
16      Beth     Fowler            6
17      Rick     Tulman           12
[('Michael', 'Smith', 6), ('Susan', 'Barker', 9), ('Robert', 'Tyler', 2), ('Susan', 'Hawthorne', 4), ('John', 'Gooding', 11), ('Helen', 'Fleming', 3), ('Chris', 'Tucker', 9), ('Sarah', 'Parker', 12), ('Jane', 'Grossman', 5), ('Paula', 'Roberts', 7), ('Thomas', 'Ziegler', 10), ('Samantha', 'Jameso